This notebook with call the necessary functions to pass a data frame to an LLM and have it make suggestions for the output:

In [2]:
#Library import for loading df
from llm_data_checker import read_df


#incase of error with file cache uncomment below command 
#importlib.reload(llm_data_checker)

data = read_df("data_test/Uncleaned_DS_jobs.csv")

#data.head()

Run this cell to check basic stats of df:

In [3]:
from llm_data_checker import df_checker

check_data = df_checker(data)


#write out the stats to external directory
with open("outputs/stats.txt", "w") as f:
    for section_name, section_value in check_data.items():
        # Write a header for this section
        f.write(f"===== {section_name} =====\n")
        
        # Write the actual data
        f.write(str(section_value))
        
        # Add spacing
        f.write("\n\n")

Anonymiser: 

Prompt builder:

In [3]:
from pathlib import Path 

#point to directoy holding prompt files
framework_txt = Path("frameworks")

#points to the directory holding the df stats 
stats_txt = Path("outputs")

#read the actual files themselves
system_template = (framework_txt / "prompt.txt").read_text()


func_test_suite = (framework_txt / "func_test_suite.txt").read_text()
function_format = (framework_txt / "function_format.txt").read_text()
stats = (stats_txt / "stats.txt").read_text()
reasoning = (framework_txt / "reasoning.txt").read_text()


#build the actual prompt 

prompt = system_template.format(
    func_test_suite=func_test_suite,
    function_format=function_format,
    stats=stats,
    reasoning=reasoning,
)

#write out the stats to external directory
with open("final_prompt/combined_prompt.txt", "w") as f:
    f.write(prompt)



In [4]:
combined_prompt = Path("final_prompt/combined_prompt.txt").read_text()
print(len(combined_prompt))

17012


API call out 


In [6]:
from cerebras.cloud.sdk import Cerebras
from pathlib import Path
import os

client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY"])

# --- Load files ---
func_test_suite = Path("frameworks/func_test_suite.txt").read_text()
function_format = Path("frameworks/function_format.txt").read_text()
stats = Path("outputs/stats.txt").read_text()
reasoning = Path("frameworks/reasoning.txt").read_text()

# --- Trim func_test_suite ---
marker = "=== APPEND NEW TRANSFORM FUNCTIONS BELOW ==="
head, _, tail = func_test_suite.partition(marker)
tail_lines = tail.splitlines()[-200:]

func_context = (
    head
    + marker
    + "\n"
    + "\n".join(tail_lines)
)

# --- System prompt (rules only) ---
system_prompt = f"""
You are an expert Python developer.

You are operating in APPEND-ONLY mode.

Rules:
- You MUST NOT modify text above the append marker.
- You MAY ONLY append new standalone functions.
- Preserve spacing, comments, and formatting.
- Each function must operate on a copy of a DataFrame.
- Leave two blank lines between appended functions.

The contents of func_test_suite.txt are authoritative.
"""

# --- User prompt (variable content) ---
user_prompt = f"""
Here is the current state of the test suite:

{func_context}

You MUST follow this function format:

{function_format}

Here are the dataset statistics:

{stats}

Use reasoning guidance from:

{reasoning}

Proceed.
"""

print("func_context chars:", len(func_context))
print("function_format chars:", len(function_format))
print("stats chars:", len(stats))
print("reasoning chars:", len(reasoning))

completion = client.chat.completions.create(
    model="llama-3.3-70b",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    max_completion_tokens=1024,
    temperature=0.0,
)

print(completion.choices[0].message.content)

func_context chars: 1327
function_format chars: 176
stats chars: 13637
reasoning chars: 172
## === APPEND NEW TRANSFORM FUNCTIONS BELOW ===


def convert_salary_estimate_to_numeric(df):
    """
    Convert the 'Salary Estimate' column to numeric values by removing the '$' and ',' characters and converting the resulting string to a float.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        tuple: A tuple containing the transformed DataFrame and a list of changes made to the DataFrame.
    """
    df_out = df.copy()
    change_log = []

    # Remove the '$' and ',' characters from the 'Salary Estimate' column
    df_out['Salary Estimate'] = df_out['Salary Estimate'].str.replace('$', '').str.replace(',', '')

    # Convert the 'Salary Estimate' column to numeric values
    df_out['Salary Estimate'] = pd.to_numeric(df_out['Salary Estimate'])

    change_log.append("Converted 'Salary Estimate' column to numeric values")

    return df_out, change_log



def extr

Write out results from LLM: